In [ ]:
import random
from IPython.display import clear_output

model.compile(loss='mse', optimizer=rms)  # reset weights of neural network
epochs = 3000
gamma = 0.975
epsilon = 1
batchSize = 40
buffer = 80
replay = []
h = 0

for i in range(epochs):
    state = initGridPlayer()
    status = 1
    while status == 1:
        qval = model.predict(state.reshape(1,64), batch_size=1)
        if random.random() < epsilon:
            action = np.random.randint(0,4)
        else:
            action = np.argmax(qval)
        new_state = makeMove(state, action)
        reward = getReward(new_state)

        if len(replay) < buffer:
            replay.append((state, action, reward, new_state))
        else:
            if h < (buffer - 1):
                h += 1
            else:
                h = 0
            replay[h] = (state, action, reward, new_state)
            minibatch = random.sample(replay, batchSize)
            X_train = []
            y_train = []
            for memory in minibatch:
                old_state, action, reward, new_state = memory
                old_qval = model.predict(old_state.reshape(1,64), batch_size=1)
                newQ = model.predict(new_state.reshape(1,64), batch_size=1)
                maxQ = np.max(newQ)
                y = np.zeros((1,4))
                y[:] = old_qval[:]
                if reward == -1:
                    update = reward + (gamma * maxQ)
                else:
                    update = reward
                y[0][action] = update
                X_train.append(old_state.reshape(64,))
                y_train.append(y.reshape(4,))

            X_train = np.array(X_train)
            y_train = np.array(y_train)
            print("Game #: %s" % (i,))
            model.fit(X_train, y_train, batch_size=batchSize, epochs=1, verbose=1)
            state = new_state
        if reward != -1:
            status = 0
        clear_output(wait=True)
    if epsilon > 0.1:
        epsilon -= (1/epochs)
